In [14]:
base_folder = "./merger proxies-20240117T025306Z-001/merger proxies/"
import os

import json

import fitz
import pandas as pd
def get_proxy_list():
    proxy_list = []
    for file in os.listdir(base_folder):
        if file.endswith(".Pdf") or file.endswith(".pdf"):
            proxy_list.append(file)
    return proxy_list

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', None)


In [3]:
print("Getting proxy list...")
proxy_list = get_proxy_list()
proxy_list[:5]

Getting proxy list...


['1Life Healthcare Inc_20220824_DEFM14A_20299948_4508791.Pdf',
 'Activision Blizzard, Inc._20220321_DEFM14A_19960426_4418687.Pdf',
 'AEROJET ROCKETDYNE HOLDINGS, INC._20210205_DEFM14A_19007640_4113036.Pdf',
 'AEROJET ROCKETDYNE HOLDINGS, INC._20230213_DEFM14A_20635156_4584450.Pdf',
 'ALKURI GLOBAL ACQUISITION CORP._20210930_DEFM14A_19561988_4303635.Pdf']

In [4]:
import os
open_ai_key = os.environ.get("OPENAI_API_KEY")
from openai import OpenAI

from openai import OpenAI
client = OpenAI(api_key=open_ai_key)
def toc_detector(all_docs_text, doc, page_number):
    response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": all_docs_text[doc][page_number]},
        {"role": "user", "content": "Table of contents page has multiple headers and page numbers in consecutive lines. Is this table of contents page. Answer yes or no only?"}
    ]
    
    )
    return response.choices[0].message.content


In [5]:
dic_page = []
all_docs_text = {}
toc_page_numbers = []
for i,file in enumerate(proxy_list):
  doc = fitz.open(base_folder+file) 
  doc_text = []
  
  for page in doc: 
    text = page.get_text() 
    doc_text.append(text)

  all_docs_text[file]=doc_text  

In [ ]:
# check whether a page is a TOC page or not for every document. 
toc_list = {}
tolerance = 1
all_docs_text2 = {}
max_toc_bound = 40
min_toc_bound = 2
for doc in all_docs_text:
    
    curr_tolerance = 0
    toc_flag_found = 0
    toc_list[doc] = []
    for page_number in range(min_toc_bound,min(max_toc_bound,len(all_docs_text[doc]))):
        resp = toc_detector(all_docs_text, doc, page_number)
        if  resp == "Yes":
            curr_tolerance = 0
            toc_flag_found = 1
            toc_list[doc].append(page_number)
            
        elif(resp=="No" and toc_flag_found==1):
            curr_tolerance += 1
            if curr_tolerance > tolerance:
                break    
    print(f"TOC pages for {doc} are {toc_list[doc]}")
      
    

In [ ]:
with open('toc_list.json', 'w') as fp:
    json.dump(toc_list, fp)

In [6]:
with open('toc_list.json', 'r') as fp:
    toc_list = json.load(fp)

In [7]:
import re
# To detect whether a given text is Roman numeral or not
def is_roman_numeral(text):
    roman_regex = r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$'
    return bool(re.match(roman_regex, text))

In [8]:
# function to filter toc pages and then extract headers and page numbers from a given TOC page
def toc_extractor(proxy_list_doc, toc_list, updated_toc_list, verbose):
    word_threshold = 30

    doc_headers_toPage = []
    save_past_number = 0
    save_counter = -1
    started_flag = 0
    updated_toc_list[proxy_list_doc] = []
    final_toc_index = -1
    toc_not_full_page_flag = 0
    if verbose:
        print(min(toc_list[proxy_list_doc]), max(toc_list[proxy_list_doc]))
    for nums in range(min(toc_list[proxy_list_doc]), max(toc_list[proxy_list_doc])+1):
        line_seperated_page = all_docs_text[proxy_list_doc][nums].strip().split("\n")
        clean_page = [x for x in line_seperated_page if x.strip()!=""]
       
 
        for i in range(len(clean_page)):
            if(clean_page[i].strip().replace(" ","").isnumeric() or is_roman_numeral(clean_page[i].strip().replace(" ","").upper())):
                save_counter = i
                header = clean_page[i-1].strip()
                current = clean_page[i].strip().replace(" ","")
                if(current.isnumeric() and str(save_past_number).isnumeric() and (int(current)<int(save_past_number))):
                    continue 
                if(started_flag and is_roman_numeral(current.upper()) and str(save_past_number).isnumeric()):
                    continue                 
                if(header!="" and current!=""):
                    doc_headers_toPage.append((header,current))
                    save_past_number = clean_page[i].strip().replace(" ","")
                    save_past_counter = i
                    started_flag = 1
                    final_toc_index = i
                    updated_toc_list[proxy_list_doc].append(nums)
                    break
                
        if save_counter==-1:
            continue
                
        for i in range(save_counter+1,len(clean_page)):
            current = clean_page[i].strip().replace(" ","")
            if verbose:
                print(current)

            if(current!=""  and current.isnumeric()):
               if verbose:
                print(current)
               if(is_roman_numeral(save_past_number.upper()) or (int(current)>=int(save_past_number) and (int(current)-int(save_past_number))<200)):
                    header = (" ".join(clean_page[save_past_counter+1:i])).strip()
                    if(header!="" and len(header.split(" "))<word_threshold and header.lower().find("annex")==-1):
                        doc_headers_toPage.append((header, current))
                        save_past_number = current
                        save_past_counter = i
                        final_toc_index = max(i, final_toc_index)
                   

            elif(current!="" and is_roman_numeral(current.upper()) and is_roman_numeral(save_past_number.upper())):
                if verbose:
                    print(current)
                header = (" ".join(clean_page[save_past_counter+1:i])).strip()
                if(header!=""  and len(header.split(" "))<word_threshold and header.lower().find("annex")==-1):
                    doc_headers_toPage.append((header, current))
                    save_past_number = current
                    save_past_counter = i
                    final_toc_index = max(i, final_toc_index)
                    
   
    df = pd.DataFrame(doc_headers_toPage, columns=["header","page"])
    return df, updated_toc_list, final_toc_index


In [9]:
# calling toc_extractor function for all documents
list_pass = proxy_list
verbose = 0
doc_to_toc = {}
updated_toc_list = {}
doc_toc_last_index_list = {}
for i in range(len(list_pass)):
    print(f"Creating TOC for {list_pass[i]}")
    df, updated_toc_list, doc_toc_last_index  = toc_extractor(list_pass[i], toc_list, updated_toc_list, verbose)
    doc_to_toc[list_pass[i]] = df
    doc_toc_last_index_list[list_pass[i]] = doc_toc_last_index

Creating TOC for 1Life Healthcare Inc_20220824_DEFM14A_20299948_4508791.Pdf
Creating TOC for Activision Blizzard, Inc._20220321_DEFM14A_19960426_4418687.Pdf
Creating TOC for AEROJET ROCKETDYNE HOLDINGS, INC._20210205_DEFM14A_19007640_4113036.Pdf
Creating TOC for AEROJET ROCKETDYNE HOLDINGS, INC._20230213_DEFM14A_20635156_4584450.Pdf
Creating TOC for ALKURI GLOBAL ACQUISITION CORP._20210930_DEFM14A_19561988_4303635.Pdf
Creating TOC for ALLEGHANY CORP DE_20220429_DEFM14A_20052995_4452227.Pdf
Creating TOC for Altra Industrial Motion Corp._20221214_DEFM14A_20506388_4559723.Pdf
Creating TOC for AMERICAN CAMPUS COMMUNITIES INC_20220616_DEFM14A_20169998_4477606.Pdf
Creating TOC for Anaplan, Inc._20220502_DEFM14A_20059827_4454121.Pdf
Creating TOC for Archaea Energy Inc._20221114_DEFM14A_20445339_4545162.Pdf
Creating TOC for ARENA PHARMACEUTICALS INC_20220103_DEFM14A_19751337_4360734.Pdf
Creating TOC for Argo Group International Holdings, Ltd._20230320_DEFM14A_20737540_4621891.Pdf
Creating TOC 

In [10]:
# function for doing the fuzy match between the headers in the TOC page and the headers in the document
def fuzzy_search_page(current_page, first_header):
    
    current_page_split = current_page.strip().split("\n")
    current_page_split = [x for x in current_page_split if x.strip()!=""]
    ret_val = None
    word_count = 0
    for j, text in enumerate(current_page_split):
        text = text.strip()
        word_count += len(text.split(" "))
        from fuzzywuzzy import fuzz
        similarity = fuzz.ratio(first_header, text)
        if(similarity>80):
            ret_val = j
            end_ind = word_count
            start_ind = end_ind - len(text.split(" "))
          
            return start_ind, end_ind
     
    current_page = current_page.replace("\n"," ")      
    if ret_val is None:
        ind = current_page.find(first_header)  
        if(ind!=-1):
            split_text = current_page[:ind + len(first_header)].strip().split(" ")
            split_text = [x for x in split_text if x.strip()!=""]
            end_ind = len(split_text)
            start_ind =  end_ind - len(first_header.strip().split(" "))
            return start_ind, end_ind

In [11]:
# finding indexes for the headers in the document using fuzzy_search_page function
from fuzzywuzzy import fuzz
doc_to_index = {}
for doc_name in proxy_list:
    digg_file_flag = 0
    print(f"Findings Indexes for headers for {doc_name}") 
    curr_df = doc_to_toc[doc_name]
    curr_doc = all_docs_text[doc_name]
    curr_lis = updated_toc_list[doc_name]
 
    stopper_flag = 0
    actual_page_lis = []
    if len(curr_lis)==0:
        doc_to_index[doc_name] = None
        continue
    st = max(curr_lis)
    for ii in curr_df.iterrows():
        
        curr_header = ii[1]["header"]
        curr_page = ii[1]["page"] 
        if not(digg_file_flag):
            if not(stopper_flag):
                for i in range(max(curr_lis), len(curr_doc)):
                    current_page_text = curr_doc[i]
                    if i==max(curr_lis):
                        temp_page = curr_doc[i].strip().split("\n")
                        temp_page = [x for x in temp_page if x.strip()!=""]
                        total_words = sum(len(x.strip().split(" ")) for x in temp_page[:doc_toc_last_index_list[doc_name]+1])
                        current_page_text = "\n".join(temp_page[doc_toc_last_index_list[doc_name]+1:])

                      
                    indices = fuzzy_search_page(current_page_text, curr_header)
                    
                    
                    if(indices!=None):
                        start_index, end_index = indices
                        if i==max(curr_lis):
                            
                            start_index = start_index + total_words
                            end_index = end_index + total_words
                          
                        actual_page_lis.append((curr_header, curr_page, i, start_index, end_index))
                        if(curr_page.isnumeric()):
                            stopper_flag = 1
                        break    

            else: 
                last_actual_page = int(actual_page_lis[-1][2])
                last_page = int(actual_page_lis[-1][1])
                curr_actual_page = last_actual_page - last_page + int(curr_page)
              
                indices = fuzzy_search_page(curr_doc[int(curr_actual_page)], curr_header)
                if(indices!=None):
                    start_index, end_index = indices
                else:
                    start_index, end_index = -1,-1

                if start_index!=-1:
                    actual_page_lis.append((curr_header, curr_page, curr_actual_page, start_index, end_index))
        if start_index==-1 or digg_file_flag:
                
                digg_file_flag = 1
                
                for i in range(st, len(curr_doc)):
                    current_page_text = curr_doc[i]
                    if i==max(curr_lis):
                        temp_page = curr_doc[i].strip().split("\n")
                        temp_page = [x for x in temp_page if x.strip()!=""]
                        current_page_text = "\n".join(temp_page[doc_toc_last_index_list[doc_name]+1:])
                    indices = fuzzy_search_page(current_page_text, curr_header)
                    if(indices!=None):
                        start_index, end_index = indices
                        if i==max(curr_lis):
                            start_index = start_index + doc_toc_last_index_list[doc_name]
                            end_index = end_index + doc_toc_last_index_list[doc_name]
                        actual_page_lis.append((curr_header, curr_page, i, start_index, end_index))
                        st = i 
                        break   
                    
    actual_page_lis_df = pd.DataFrame(actual_page_lis, columns=["header","page","actual_page", "start_index", "end_index"])
    doc_to_index[doc_name] = actual_page_lis_df
        
    
            




Findings Indexes for headers for 1Life Healthcare Inc_20220824_DEFM14A_20299948_4508791.Pdf


c:\Users\parth\OneDrive\Documents\Harvey\.conda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Findings Indexes for headers for Activision Blizzard, Inc._20220321_DEFM14A_19960426_4418687.Pdf
Findings Indexes for headers for AEROJET ROCKETDYNE HOLDINGS, INC._20210205_DEFM14A_19007640_4113036.Pdf
Findings Indexes for headers for AEROJET ROCKETDYNE HOLDINGS, INC._20230213_DEFM14A_20635156_4584450.Pdf
Findings Indexes for headers for ALKURI GLOBAL ACQUISITION CORP._20210930_DEFM14A_19561988_4303635.Pdf
Findings Indexes for headers for ALLEGHANY CORP DE_20220429_DEFM14A_20052995_4452227.Pdf
Findings Indexes for headers for Altra Industrial Motion Corp._20221214_DEFM14A_20506388_4559723.Pdf
Findings Indexes for headers for AMERICAN CAMPUS COMMUNITIES INC_20220616_DEFM14A_20169998_4477606.Pdf
Findings Indexes for headers for Anaplan, Inc._20220502_DEFM14A_20059827_4454121.Pdf
Findings Indexes for headers for Archaea Energy Inc._20221114_DEFM14A_20445339_4545162.Pdf
Findings Indexes for headers for ARENA PHARMACEUTICALS INC_20220103_DEFM14A_19751337_4360734.Pdf
Findings Indexes for hea

In [12]:
# creating paragraphs for each section in the document
doc_to_para = {}
for doc_name in proxy_list:
    print(f"Creating paragraphs for {doc_name}")
    curr_df = doc_to_index[doc_name]
    curr_doc = all_docs_text[doc_name]
    curr_lis = updated_toc_list[doc_name]
    if len(curr_lis)==0:
        doc_to_para[doc_name] = None
        continue
    para_lis = []
  
    for i in range(len(curr_df)-1):
        page_number = curr_df.iloc[i]["actual_page"]
        start_index = curr_df.iloc[i]["start_index"]
        end_index = curr_df.iloc[i]["end_index"]
        next_page_number = curr_df.iloc[i+1]["actual_page"]
        next_start_index = curr_df.iloc[i+1]["start_index"]
    

        section_body = ""
        if page_number==next_page_number:
            current_page_text = curr_doc[page_number].replace("\n"," ").split(" ")
            current_page_text = [x for x in current_page_text if x.strip()!=""][end_index:next_start_index]
            section_body += " ".join(current_page_text)
        else:
            for j in range(page_number, next_page_number+1):
                if j==page_number:
                    current_page_text = curr_doc[j].replace("\n"," ").split(" ")
                    current_page_text = [x for x in current_page_text if x.strip()!=""][end_index:]
                elif j==next_page_number:
                    current_page_text = curr_doc[j].replace("\n"," ").split(" ")[:next_start_index]
                    current_page_text = [x for x in current_page_text if x.strip()!=""]
                else:
                    current_page_text = curr_doc[j].replace("\n"," ").split(" ")
                    current_page_text = [x for x in current_page_text if x.strip()!=""]

                section_body += " ".join(current_page_text)
        section_body = section_body.replace("Table of Contents","").replace("Table of Content","").replace("TABLE OF CONTENTS","").replace("TABLE OF CONTENT","")        
        para_lis.append(section_body)
   
    page_number = curr_df.iloc[-1]["actual_page"]
    end_index = curr_df.iloc[-1]["end_index"]
    section_body = ""
    
    for j in range(page_number, len(curr_doc)):
        if j==page_number:
            current_page_text = curr_doc[j].replace("\n"," ").split(" ")
            current_page_text = [x for x in current_page_text if x.strip()!=""][end_index:]
        else:
            current_page_text = curr_doc[j].replace("\n"," ").split(" ")
            current_page_text = [x for x in current_page_text if x.strip()!=""]
        section_body += " ".join(current_page_text)
    section_body = section_body.replace("Table of Contents","").replace("Table of Content","").replace("TABLE OF CONTENTS","").replace("TABLE OF CONTENT","")        
    para_lis.append(section_body)
    

    
    curr_df["section_body"] = para_lis
    doc_to_para[doc_name] = curr_df


    

Creating paragraphs for 1Life Healthcare Inc_20220824_DEFM14A_20299948_4508791.Pdf
Creating paragraphs for Activision Blizzard, Inc._20220321_DEFM14A_19960426_4418687.Pdf
Creating paragraphs for AEROJET ROCKETDYNE HOLDINGS, INC._20210205_DEFM14A_19007640_4113036.Pdf
Creating paragraphs for AEROJET ROCKETDYNE HOLDINGS, INC._20230213_DEFM14A_20635156_4584450.Pdf
Creating paragraphs for ALKURI GLOBAL ACQUISITION CORP._20210930_DEFM14A_19561988_4303635.Pdf
Creating paragraphs for ALLEGHANY CORP DE_20220429_DEFM14A_20052995_4452227.Pdf
Creating paragraphs for Altra Industrial Motion Corp._20221214_DEFM14A_20506388_4559723.Pdf
Creating paragraphs for AMERICAN CAMPUS COMMUNITIES INC_20220616_DEFM14A_20169998_4477606.Pdf
Creating paragraphs for Anaplan, Inc._20220502_DEFM14A_20059827_4454121.Pdf
Creating paragraphs for Archaea Energy Inc._20221114_DEFM14A_20445339_4545162.Pdf
Creating paragraphs for ARENA PHARMACEUTICALS INC_20220103_DEFM14A_19751337_4360734.Pdf
Creating paragraphs for Argo Gr

In [13]:
doc_to_para_final = {}
for doc_name in proxy_list:
    
    print(f"Creating document name column for {doc_name}")
    curr_df = doc_to_para[doc_name]
    if curr_df is None:
        continue
    current = curr_df.copy()
    current["document_name"] = doc_name
    current = current.rename(columns={"document_name":"Document", "header":"Section Header", "page":"Page Number", "section_body":"Section Text"})
    current = current[["Document", "Section Header", "Section Text", "Page Number"]]
    doc_to_para_final[doc_name] = current


Creating document name column for 1Life Healthcare Inc_20220824_DEFM14A_20299948_4508791.Pdf
Creating document name column for Activision Blizzard, Inc._20220321_DEFM14A_19960426_4418687.Pdf
Creating document name column for AEROJET ROCKETDYNE HOLDINGS, INC._20210205_DEFM14A_19007640_4113036.Pdf
Creating document name column for AEROJET ROCKETDYNE HOLDINGS, INC._20230213_DEFM14A_20635156_4584450.Pdf
Creating document name column for ALKURI GLOBAL ACQUISITION CORP._20210930_DEFM14A_19561988_4303635.Pdf
Creating document name column for ALLEGHANY CORP DE_20220429_DEFM14A_20052995_4452227.Pdf
Creating document name column for Altra Industrial Motion Corp._20221214_DEFM14A_20506388_4559723.Pdf
Creating document name column for AMERICAN CAMPUS COMMUNITIES INC_20220616_DEFM14A_20169998_4477606.Pdf
Creating document name column for Anaplan, Inc._20220502_DEFM14A_20059827_4454121.Pdf
Creating document name column for Archaea Energy Inc._20221114_DEFM14A_20445339_4545162.Pdf
Creating document n

In [38]:
for doc_name in proxy_list:
    if doc_name not in  doc_to_para_final:
        continue
    doc_to_para_final[doc_name].to_csv(f"final/{doc_name}.csv", index=False)

In [15]:
base_prompt = '''
ermination
Termination for Cause: "Either party may terminate this Agreement immediately upon written notice if the other party breaches any material term or condition of this Agreement and fails to cure such breach within 30 days after receipt of written notice of the same."

Termination Upon Notice: "This Agreement may be terminated by either party upon 90 days' written notice to the other party."

Automatic Termination: "This Agreement shall automatically terminate upon the completion of the services set forth in Exhibit A."

Termination for Convenience: "The Client may terminate this Agreement at any time without cause by providing 30 days written notice to the Service Provider."

Termination upon Bankruptcy: "Either party may terminate this Agreement immediately if the other party becomes insolvent, files for bankruptcy, or goes into receivership."

Termination for Non-payment: "In the event of non-payment of any fees due under this Agreement, the Service Provider may terminate this Agreement upon 15 days' written notice."

Mutual Termination: "This Agreement may be terminated by mutual agreement of both parties in writing."

Termination by Law: "This Agreement will terminate automatically without notice if it becomes illegal for the parties to continue."

Termination upon Death or Incapacity: "This Agreement shall terminate automatically upon the death or incapacity of the Consultant."

Survival of Termination: "Upon termination of this Agreement, Sections X, Y, and Z shall survive and continue in full force and effect."

Indemnification
General Indemnification: "The Contractor shall indemnify and hold harmless the Client against any and all claims, damages, liabilities, and expenses arising from the Contractor's performance or nonperformance of its obligations under this Agreement."

Third-Party Claims: "The Service Provider agrees to indemnify, defend, and hold harmless the Client from any claims, damages, or expenses, including reasonable attorney's fees, brought by a third party resulting from the Service Provider's work under this Agreement."

Product Liability: "The Manufacturer agrees to indemnify the Distributor against any losses or damages arising from defects in the products supplied under this Agreement."

Intellectual Property Indemnification: "The Software Developer will indemnify and defend the Client against any infringement claims related to the software provided under this Agreement."

Employee Action: "The Employer shall indemnify and hold harmless the Employee against any claims or legal actions arising from the Employee's lawful actions during the course of their employment."

Environmental Indemnification: "The Seller agrees to indemnify and hold the Buyer harmless from any claims arising from environmental contamination found on the property."

Breach of Agreement: "The Party in breach shall indemnify and hold harmless the other Party from damages resulting from said breach."

Data Breach: "In the event of a data breach, the Data Processor shall indemnify the Data Controller for any damages incurred due to the breach."

Professional Negligence: "The Consultant shall indemnify the Client for any losses or damages resulting from professional negligence or willful misconduct in the performance of the Consultant's duties."

Insurance Claims: "The Insured agrees to indemnify the Insurer against any false claims or misrepresentations made under this policy."

Confidentiality
Basic Confidentiality Clause: "The Receiving Party agrees to keep confidential all proprietary information disclosed by the Disclosing Party and to use such information only for the purposes of this Agreement."

Non-Disclosure: "Each party agrees not to disclose any confidential information provided by the other party to any third parties without prior written consent."

Return or Destruction of Confidential Information: "Upon termination of this Agreement, the Receiving Party shall return or destroy all copies of the Confidential Information."

Exclusions from Confidential Information: "Information that is already in the public domain or becomes publicly known through no fault of the Receiving Party shall not be considered confidential."

Duration of Confidentiality Obligations: "The obligations of confidentiality under this Agreement shall survive for a period of five years after the termination of this Agreement."

Use of Confidential Information: "The Receiving Party may use the Confidential Information solely for the purpose of fulfilling its obligations under this Agreement."

Confidentiality in Employee Agreements: "The Employee agrees not to disclose any confidential information obtained during the course of employment to any third party, both during and after the term of their employment."

Breach of Confidentiality: "Any breach of the confidentiality obligations may result in immediate legal action for injunctive relief and damages."

Confidentiality in Joint Ventures: "Parties to this Joint Venture Agreement agree to maintain the confidentiality of all proprietary information shared in the course of the venture."

Exceptions for Legal Disclosure: "The Receiving Party may disclose Confidential Information if required by law or court order, provided that the Disclosing Party is given prompt notice and the disclosure is limited to the required extent."

'''

In [18]:
def classify_section(text):
    response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": base_prompt + "\n"+ "Now consider the following text" +"\n" + text},
        {"role": "user", "content": "Classify this text into one of the following categories: Termination, Indemnification, Confidentiality. If you think it belongs to none of these categories, return None. Output should be one of the following: Termination, Indemnification, Confidentiality, None."}
    ]
    
    )
    return response.choices[0].message.content
    

In [24]:
current_df = doc_to_para_final["1Life Healthcare Inc_20220824_DEFM14A_20299948_4508791.Pdf"]
for i in range(len(current_df)):
    text = current_df.iloc[i]["Section Text"]
    text_for_gpt = " ".join(text.split(" ")[:4000])
    header = current_df.iloc[i]["Section Header"]
    class_section = classify_section(header + "\n" +text_for_gpt)
    if class_section in ["Termination", "Indemnification", "Confidentiality","None"]:
        print(f"The section {header} belongs to {class_section}")
    else:
        print(f"The section {header} belongs to None")    


The section SUMMARY belongs to None
The section The Special Meeting belongs to None
The section Shares Held by 1Life’s Directors and Executive Officers belongs to None
The section The Merger belongs to Termination
The section Merger Consideration belongs to None
The section Recommendation of the 1Life Board of Directors belongs to Termination
The section Opinion of Morgan Stanley & Co. LLC belongs to None
The section Interests of 1Life’s Directors and Executive Officers in the Merger belongs to None
The section Appraisal Rights belongs to None
The section Material U.S. Federal Income Tax Consequences of the Merger belongs to None
The section Regulatory Approvals Required for the Merger belongs to None
The section Conduct of Business Pending the Merger belongs to Termination
The section Conditions to the Closing of the Merger belongs to None
The section Termination of the Merger Agreement belongs to Termination
The section QUESTIONS AND ANSWERS belongs to None
The section FORWARD-LOOKIN